In [1]:
import pymysql
import requests
import pandas as pd

### 下載分析pm2.5 opendata資料

In [29]:
url="https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=4c89a32a-a214-461b-bf29-30ff32a61a8a&limit=1000&sort=datacreationdate%20desc&format=JSON"
url

'https://data.moenv.gov.tw/api/v2/aqx_p_02?api_key=4c89a32a-a214-461b-bf29-30ff32a61a8a&limit=1000&sort=datacreationdate%20desc&format=JSON'

In [30]:
res=requests.get(url,verify=False)
res

c:\Users\roger\Desktop\Django-陳岳洋\5.MYSQL\pm25_tomysql\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'data.moenv.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [31]:
datas=res.json()["records"]
datas

[{'site': '林森',
  'county': '臺南市',
  'pm25': '18',
  'datacreationdate': '2025-12-06 17:00',
  'itemunit': 'μg/m3'},
 {'site': '員林',
  'county': '彰化縣',
  'pm25': '14',
  'datacreationdate': '2025-12-06 17:00',
  'itemunit': 'μg/m3'},
 {'site': '大城',
  'county': '彰化縣',
  'pm25': '15',
  'datacreationdate': '2025-12-06 17:00',
  'itemunit': 'μg/m3'},
 {'site': '富貴角',
  'county': '新北市',
  'pm25': '10',
  'datacreationdate': '2025-12-06 17:00',
  'itemunit': 'μg/m3'},
 {'site': '麥寮',
  'county': '雲林縣',
  'pm25': '12',
  'datacreationdate': '2025-12-06 17:00',
  'itemunit': 'μg/m3'},
 {'site': '關山',
  'county': '臺東縣',
  'pm25': '5',
  'datacreationdate': '2025-12-06 17:00',
  'itemunit': 'μg/m3'},
 {'site': '馬公',
  'county': '澎湖縣',
  'pm25': '10',
  'datacreationdate': '2025-12-06 17:00',
  'itemunit': 'μg/m3'},
 {'site': '金門',
  'county': '金門縣',
  'pm25': '13',
  'datacreationdate': '2025-12-06 17:00',
  'itemunit': 'μg/m3'},
 {'site': '馬祖',
  'county': '連江縣',
  'pm25': '17',
  'datacreati

In [32]:
len(datas)

1000

In [33]:
datas[0]

{'site': '林森',
 'county': '臺南市',
 'pm25': '18',
 'datacreationdate': '2025-12-06 17:00',
 'itemunit': 'μg/m3'}

### 建立資料表

In [34]:
table_str="""
create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
count varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)
"""
print(table_str)


create table if not exists pm25(
id int auto_increment primary key,
site varchar(25),
count varchar(50),
pm25 int,
datacreationdate datetime,
itemunit varchar(20),
unique key site_time (site,datacreationdate)
)



### 連接資料庫

In [ ]:
import os
from dotenv import load_dotenv
conn=pymysql.connect(
    host="mysql-12b99084-pm25.c.aivencloud.com",
    user="avnadmin",
    password=os.getenv("DB_PASSWORD"),
    port=21697,
    database="defaultdb"
)

cursor=conn.cursor()
cursor


In [36]:
cursor.execute(table_str)
conn.commit()

### 寫入一筆資料

In [37]:
sqlstr="insert into pm25 (site,count,pm25,datacreationdate,itemunit)\
    values('{}','{}',{},'{}','{}')" 
data=list(datas[0].values())
print(data)
sqlstr.format(data[0],data[1],data[2],data[3],data[4])


['林森', '臺南市', '18', '2025-12-06 17:00', 'μg/m3']


"insert into pm25 (site,count,pm25,datacreationdate,itemunit)    values('林森','臺南市',18,'2025-12-06 17:00','μg/m3')"

In [38]:
cursor.execute(sqlstr.format(data[0],data[1],data[2],data[3],data[4]))
conn.commit()

In [39]:
datas[0].values()

dict_values(['林森', '臺南市', '18', '2025-12-06 17:00', 'μg/m3'])

### 一次寫入
- 加上ignore

In [ ]:
sqlstr="insert ignore into pm25 (site,count,pm25,datacreationdate,itemunit)\
    values(%s,%s,%s,%s,%s)"

# 移除pm25空字串部分
values=[list(data.values()) for data in datas if list(data.values())[2]!=""]
values
cursor.executemany(sqlstr,values)
conn.commit()

TypeError: not all arguments converted during string formatting

: 

In [26]:
conn.close()

In [25]:
cursor.execute("select * from pm25")

datas=cursor.fetchall()

datas

((1, '員林', '彰化縣', 9, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (2, '大城', '彰化縣', 11, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (3, '富貴角', '新北市', 9, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (4, '麥寮', '雲林縣', 15, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (5, '關山', '臺東縣', 4, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (6, '馬公', '澎湖縣', 16, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (7, '金門', '金門縣', 15, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (8, '馬祖', '連江縣', 12, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (9, '埔里', '南投縣', 29, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (10, '復興', '高雄市', 33, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (11, '永和', '新北市', 11, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (12, '中壢', '桃園市', 13, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (13, '三重', '新北市', 12, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (14, '冬山', '宜蘭縣', 5, datetime.datetime(2025, 12, 5, 14, 0), 'μg/m3'),
 (15

In [96]:
df=pd.DataFrame(datas,columns=["id","site","county","pm25","datetime","unit"])
df

,id,site,county,pm25,datetime,unit
0,1,員林,彰化縣,21,2025-12-02 01:00:00,μg/m3
1,2,大城,彰化縣,9,2025-12-02 01:00:00,μg/m3
2,3,富貴角,新北市,4,2025-12-02 01:00:00,μg/m3
3,4,麥寮,雲林縣,6,2025-12-02 01:00:00,μg/m3
4,5,關山,臺東縣,7,2025-12-02 01:00:00,μg/m3
...,...,...,...,...,...,...
1924,5801,新竹,新竹市,3,2025-12-02 04:00:00,μg/m3
1925,5802,竹東,新竹縣,3,2025-12-02 04:00:00,μg/m3
1926,5803,湖口,新竹縣,7,2025-12-02 04:00:00,μg/m3
1927,5804,龍潭,桃園市,5,2025-12-02 04:00:00,μg/m3


In [104]:
df.groupby("county").get_group("新北市").drop_duplicates()

,id,site,county,pm25,datetime,unit
2,3,富貴角,新北市,4,2025-12-02 01:00:00,μg/m3
9,10,永和,新北市,5,2025-12-02 01:00:00,μg/m3
12,13,三重,新北市,6,2025-12-02 01:00:00,μg/m3
68,69,淡水,新北市,4,2025-12-02 01:00:00,μg/m3
69,70,林口,新北市,0,2025-12-02 01:00:00,μg/m3
...,...,...,...,...,...,...
1867,5744,新店,新北市,8,2025-12-02 05:00:00,μg/m3
1868,5745,汐止,新北市,2,2025-12-02 05:00:00,μg/m3
1872,5749,富貴角,新北市,10,2025-12-02 04:00:00,μg/m3
1879,5756,永和,新北市,5,2025-12-02 04:00:00,μg/m3


In [99]:
len(df)

1929

In [102]:
df[["site","datetime"]].drop_duplicates()

,site,datetime
0,員林,2025-12-02 01:00:00
1,大城,2025-12-02 01:00:00
2,富貴角,2025-12-02 01:00:00
3,麥寮,2025-12-02 01:00:00
4,關山,2025-12-02 01:00:00
...,...,...
1924,新竹,2025-12-02 04:00:00
1925,竹東,2025-12-02 04:00:00
1926,湖口,2025-12-02 04:00:00
1927,龍潭,2025-12-02 04:00:00


### 取得各縣市站點的平均值

In [10]:
cursor.execute("select avg(pm25) from pm25;")
float(cursor.fetchone()[0])

14.2253

In [15]:
sqlstr="""
select count,round(avg(pm25),2) from pm25 group by count;
"""

cursor.execute(sqlstr)
cursor.fetchall()

(('彰化縣', Decimal('16.24')),
 ('新北市', Decimal('8.24')),
 ('雲林縣', Decimal('20.29')),
 ('臺東縣', Decimal('6.01')),
 ('澎湖縣', Decimal('10.73')),
 ('金門縣', Decimal('17.09')),
 ('連江縣', Decimal('13.73')),
 ('南投縣', Decimal('20.49')),
 ('高雄市', Decimal('21.05')),
 ('桃園市', Decimal('7.95')),
 ('宜蘭縣', Decimal('7.06')),
 ('臺北市', Decimal('7.61')),
 ('花蓮縣', Decimal('6.27')),
 ('屏東縣', Decimal('18.03')),
 ('臺南市', Decimal('21.59')),
 ('嘉義市', Decimal('22.98')),
 ('嘉義縣', Decimal('20.77')),
 ('臺中市', Decimal('15.49')),
 ('苗栗縣', Decimal('11.34')),
 ('新竹市', Decimal('7.49')),
 ('新竹縣', Decimal('8.87')),
 ('基隆市', Decimal('7.90')))

In [17]:
conn.close()